## coverting wikipedia table into dataframe

In [68]:
from bs4 import BeautifulSoup as bsoup
import lxml
import pandas as pd
from pandas import DataFrame
import numpy as np
import requests as req
from urllib.request import urlopen as ureq
print("done!")

done!


In [6]:
my_url= 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [7]:
r= req.get(my_url)

## Parsing the web html file with BeautifulSoup packag

In [8]:
page=bsoup(r.text,"html.parser")
page

<!DOCTYPE html>

<html class="client-nojs" dir="ltr" lang="en">
<head>
<meta charset="utf-8"/>
<title>List of postal codes of Canada: M - Wikipedia</title>
<script>document.documentElement.className="client-js";RLCONF={"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of postal codes of Canada: M","wgCurRevisionId":920980179,"wgRevisionId":920980179,"wgArticleId":539066,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Communications in Ontario","Postal codes in Canada","Toronto","Ontario-related lists"],"wgBreakFrames":!1,"wgPageContentLanguage":"en","wgPageContentModel":"wikitext","wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgMonthNamesShort":["","Jan",

## finding the table into the page

In [13]:
table=page.table
table

<table class="wikitable sortable">
<tbody><tr>
<th>Postcode</th>
<th>Borough</th>
<th>Neighbourhood
</th></tr>
<tr>
<td>M1A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M2A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M3A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Parkwoods" title="Parkwoods">Parkwoods</a>
</td></tr>
<tr>
<td>M4A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Victoria_Village" title="Victoria Village">Victoria Village</a>
</td></tr>
<tr>
<td>M5A</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Harbourfront_(Toronto)" title="Harbourfront (Toronto)">Harbourfront</a>
</td></tr>
<tr>
<td>M5A</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Regent_Park" title="Regent Park">Regent Park</a>
</td></tr>
<tr>
<td>M6A</td>

In [36]:
 results=table.find_all('tr')
rows=len(results)
rows

289

In [37]:
results[0:5]

[<tr>
 <th>Postcode</th>
 <th>Borough</th>
 <th>Neighbourhood
 </th></tr>, <tr>
 <td>M1A</td>
 <td>Not assigned</td>
 <td>Not assigned
 </td></tr>, <tr>
 <td>M2A</td>
 <td>Not assigned</td>
 <td>Not assigned
 </td></tr>, <tr>
 <td>M3A</td>
 <td><a href="/wiki/North_York" title="North York">North York</a></td>
 <td><a href="/wiki/Parkwoods" title="Parkwoods">Parkwoods</a>
 </td></tr>, <tr>
 <td>M4A</td>
 <td><a href="/wiki/North_York" title="North York">North York</a></td>
 <td><a href="/wiki/Victoria_Village" title="Victoria Village">Victoria Village</a>
 </td></tr>]

## The Header of the DataFrame is the first row of data

In [38]:
header=results[0].text.split()
header

['Postcode', 'Borough', 'Neighbourhood']

## Let's check some rows in order to prepare to build the loop that will extract all cells into a DataFrame. For example, let's examine row 100

In [39]:
text=results[100].text.split('\n')
text

['', 'M5K', 'Downtown Toronto', 'Design Exchange', '']

In [40]:
Postcode=text[1]
Postcode

'M5K'

In [41]:
Borough=text[2]
Borough

'Downtown Toronto'

In [42]:
Neighborhood=text[3]
Neighborhood

'Design Exchange'

## Iteration loop to extract all cells into a dataframe df

In [47]:
records =[]
n=1
while n < rows :
    Postcode=results[n].text.split('\n')[1]
    Borough=results[n].text.split('\n')[2]
    Neighborhood=results[n].text.split('\n')[3]
    records.append((Postcode, Borough,Neighborhood))
    n=n+1

df=pd.DataFrame(records, columns=['PostalCode', 'Borough', 'Neighbourhood'])
df.head(5)

PostalCode           Borough     Neighbourhood
0        M1A      Not assigned      Not assigned
1        M2A      Not assigned      Not assigned
2        M3A        North York         Parkwoods
3        M4A        North York  Victoria Village
4        M5A  Downtown Toronto      Harbourfront

In [48]:
df.shape

(288, 3)

In [49]:
df.tail()

PostalCode       Borough          Neighbourhood
283        M8Z     Etobicoke              Mimico NW
284        M8Z     Etobicoke     The Queensway West
285        M8Z     Etobicoke  Royal York South West
286        M8Z     Etobicoke         South of Bloor
287        M9Z  Not assigned           Not assigned

## there are some rows with not assigned strings

In [50]:
df[df['Borough']=='Not assigned'].count()

PostalCode       77
Borough          77
Neighbourhood    77
dtype: int64

In [51]:
df1=df[~df.Borough.str.contains("Not assigned")]                          #to drop those rows
df1=df1.reset_index(drop=True)

In [56]:
df1.shape

(211, 3)

## Let's examine the dataframe first to set up the best loop algorithm. How many unique postalcodes , Boroughs and Neighbourhood names are there?

In [59]:
postalcodes = df1['PostalCode'].nunique()
boroughs = df1['Borough'].nunique()
neighbourhoods= df1['Neighbourhood'].nunique()
print('Unique Postalcodes : ' + str(postalcodes))
print('Unique Boroughs  : '+ str(boroughs))
print('Unique Neighbourhoods  :' + str(neighbourhoods))

Unique Postalcodes : 103
Unique Boroughs  : 11
Unique Neighbourhoods  :209


In [60]:
df1.head()

PostalCode           Borough     Neighbourhood
0        M3A        North York         Parkwoods
1        M4A        North York  Victoria Village
2        M5A  Downtown Toronto      Harbourfront
3        M5A  Downtown Toronto       Regent Park
4        M6A        North York  Lawrence Heights

In [61]:
rows1=len(df1)
rows1

211

In [62]:
df2=df1
df2.head()      #copy of data frame

PostalCode           Borough     Neighbourhood
0        M3A        North York         Parkwoods
1        M4A        North York  Victoria Village
2        M5A  Downtown Toronto      Harbourfront
3        M5A  Downtown Toronto       Regent Park
4        M6A        North York  Lawrence Heights

In [64]:
rows2=len(df2)-1
rows2

210

In [65]:
n=0

while n < nrows2 :
    post1=df2.iloc[n,0]
    m=n+1
    post2=df2.iloc[m,0]
    neigh1=df2.iloc[n,2]
    neigh2=df2.iloc[m,2]
    if post1==post2:
        df2.Neighbourhood[n,2] = neigh1=neigh1+','+neigh2
        df2=df2.drop(df2.index[m])
        nrows2=nrows2-1
        df2 = df2.reset_index(drop=True)
    else:
        n=n+1
df2.index

RangeIndex(start=0, stop=103, step=1)

In [66]:
df2.shape

(103, 3)

In [67]:
df2.head()

PostalCode           Borough  \
0        M3A        North York   
1        M4A        North York   
2        M5A  Downtown Toronto   
3        M6A        North York   
4        M7A      Queen's Park   

                                       Neighbourhood  
0                                          Parkwoods  
1                                   Victoria Village  
2  Kingsway Park South West,Mimico NW,The Queensw...  
3                    Lawrence Heights,Lawrence Manor  
4                                       Not assigned